In [32]:
import numpy as np
from collections import Counter

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
        self.feature = feature # index of the feature to be tested
        self.threshold = threshold  # value at which we will do the split
        self.left = left # left child
        self.right = right # right child
        self.value = value # value if the node is a leaf in the tree
        
    def is_leaf_node(self):
        return self.value is not None
    
    def disp(self):
        print(f"Node: \t\nfeature",self.feature, "\t\ntreshold", self.threshold,"\t\nvalue", self.value, "    \nleft", self.left, "    \nright", self.right, sep='\t')
        if self.left is not None:
            print("LEFT side")
            self.left.disp()
        if self.right is not None:
            print("RIGHT side")
            self.right.disp()


class DecisionTree:
    #max_depth: REQUIRED
    #min_samples_split: REQUIRED
    #min_samples_leaf: REQUIRED
    def __init__(self, max_depth=10, min_samples_split=2, min_samples_leaf = 1, n_features=None, random_state=None):
        self.min_samples_split=min_samples_split # minimum number of samples required to split an internal node
        self.max_depth=max_depth # maximum depth of the tree
        self.n_features=n_features # number of features to consider when looking for the best split
        self.min_samples_leaf=min_samples_leaf # minimum number of samples required to be at a leaf node
        self.random_state=random_state # random number generator seed for reproducibility
        self.root=None

    #For training the model
    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1],self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        #recursively split on the best split
        n_samples, n_feats = X.shape
        n_labels = len(np.unique(y))

        # check the stopping criteria
        if (depth>=self.max_depth or n_labels==1 or n_samples<self.min_samples_split):
            # if any of the above condition is true, then we are at a leaf node
            # so determine the value of the leaf node
            # value is the most common class label
            # return the node
            #TODO: other stopping criteria
            leaf_value = self._most_common_label(y)
            #print("Kirtérium elérve:",depth)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_feats, self.n_features, replace=False)

        # find the best split
        best_feature, best_thresh = self._best_split(X, y, feat_idxs)

        # create child nodes
        left_idxs, right_idxs = self._split(X[:, best_feature], best_thresh)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
        return Node(feature=best_feature, threshold=best_thresh, left=left, right=right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        # parent entropy
        parent_entropy = self._entropy(y)

        # create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        
        # calculate the weighted avg. entropy of children
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_l/n) * e_l + (n_r/n) * e_r

        # calculate the IG
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        hist = np.bincount(y)
        ps = hist / len(y)
        return -np.sum([p * np.log(p) for p in ps if p>0])

    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    #For predicting the output
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            #print("Levél value: ", node.value, "treshold: ",node.threshold, "feature: ",node.feature)
            return node.value

        if x[node.feature] <= node.threshold:
            #print("Csomopont")
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)
    
    #For printing the tree
    def print(self):
        self.root.disp()
        pass


In [33]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn import tree
import matplotlib.pyplot as plt

#Minta adatforrás
wine = datasets.load_breast_cancer()
X = wine.data # adatok
y = wine.target # címkék/célváltozók

#Adatok felosztása tanító és tesztelő halmazra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) # 80% tanító, 20% tesztelő

#Döntési fa tanítása
dt = DecisionTree(max_depth=3)
dt.fit(X_train, y_train)

#Döntési fa kiértékelése
y_pred = dt.predict(X_test)

def accuracy(y_test, y_pred):
    return np.sum(y_test == y_pred) / len(y_test)

acc = accuracy(y_test, y_pred)
print("Accuracy: ", acc)

dt.print()

Accuracy:  0.9473684210526315
Node: 	
feature	27		
treshold	0.1423		
value	None	    
left	<__main__.Node object at 0x13d3baa50>	    
right	<__main__.Node object at 0x13d3bbc90>
LEFT side
Node: 	
feature	22		
treshold	107.4		
value	None	    
left	<__main__.Node object at 0x13c9b9b50>	    
right	<__main__.Node object at 0x13d3ba8d0>
LEFT side
Node: 	
feature	18		
treshold	0.01647		
value	None	    
left	<__main__.Node object at 0x111a8d050>	    
right	<__main__.Node object at 0x13ce67ed0>
LEFT side
Node: 	
feature	None		
treshold	None		
value	1	    
left	None	    
right	None
RIGHT side
Node: 	
feature	None		
treshold	None		
value	1	    
left	None	    
right	None
RIGHT side
Node: 	
feature	21		
treshold	19.58		
value	None	    
left	<__main__.Node object at 0x13ce180d0>	    
right	<__main__.Node object at 0x13ce1b010>
LEFT side
Node: 	
feature	None		
treshold	None		
value	1	    
left	None	    
right	None
RIGHT side
Node: 	
feature	None		
treshold	None		
value	0	    
left	None	    
right	Non